In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='0'

import torch

print(torch.__version__)
print(torch.version.cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name)

2.1.0
11.8
cuda
True
0
<function get_device_name at 0x7fe1836485e0>


In [ ]:
import pandas as pd
import os
data_path = '../../data/jeehoshin/foodcom_dataset/'

raw_interaction = pd.read_csv(data_path + 'RAW_interactions.csv')
pp_recipe = pd.read_csv(data_path + 'PP_recipes.csv')
raw_recipe = pd.read_csv(data_path + 'RAW_recipes.csv')

records_removed = True
df = raw_interaction[raw_interaction['recipe_id'].isin(pp_recipe['id'].tolist())].reset_index(drop=True)

while records_removed:
    user_counts = df['user_id'].value_counts()
    item_counts = df['recipe_id'].value_counts()

    valid_users = user_counts[user_counts >= 5].index
    valid_items = item_counts[item_counts >= 5].index

    filtered_df = df[df['user_id'].isin(valid_users) & df['recipe_id'].isin(valid_items)].reset_index(drop=True)

    if len(filtered_df) == len(df):
        records_removed = False
    else:
        records_removed = True

    df = filtered_df
    
core_inter = df
sort_filter_ui = core_inter.sort_values('date').reset_index(drop=True)

file_names = os.listdir(data_path + 'images/')

finish_image = [int(os.path.splitext(file_name)[0]) for file_name in file_names]
sort_filter_ui = sort_filter_ui[sort_filter_ui['recipe_id'].isin(finish_image)].reset_index(drop=True)
print(len(finish_image))
sort_filter_ui

29943


,user_id,recipe_id,date,rating,review
0,2999,3567,2000-10-23,5,I have made this pie instead of plain ol' pump...
1,2178,3704,2000-10-30,3,Careful not to cook it too long... you want th...
2,2178,4366,2000-11-04,5,"if you like oysters, this is a great alternati..."
3,5523,7695,2001-02-01,1,I agree.
4,42189,4460,2001-02-12,5,I have had this before. It has really good fla...
...,...,...,...,...,...
428496,2001513060,367414,2018-12-17,1,"maybe I did something wrong , but I thought th..."
428497,2001513060,192495,2018-12-17,5,This is a keeper. Delicious Soup that both my ...
428498,454804,20713,2018-12-17,0,"Made this as gifts. Did 6, quart jars plus had..."
428499,1290903,131607,2018-12-18,5,This is a great recipe for a nice thin crispy ...


In [ ]:
print(f"# of recipes with images : {len(file_names)}")

users = sort_filter_ui['user_id'].unique().tolist()
items = sort_filter_ui['recipe_id'].unique().tolist()
print(f"number of users: {len(users)}, number of items: {len(items)}")

train_ = sort_filter_ui[:int(0.6*len(sort_filter_ui))]
valid_ = sort_filter_ui[int(0.6*len(sort_filter_ui)):int(0.7*len(sort_filter_ui))]
test_ = sort_filter_ui[int(0.7*len(sort_filter_ui)):]

u_tr = set(train_['user_id'].tolist())
u_va = set(valid_['user_id'].tolist())
u_te = set(test_['user_id'].tolist())
u_total = u_tr & u_te

filter_u_tr = train_[train_['user_id'].isin(u_total)].reset_index(drop=True)
filter_u_te = test_[test_['user_id'].isin(u_total)].reset_index(drop=True)
filter_u_va = valid_[valid_['user_id'].isin(u_total)].reset_index(drop=True)
print(f"train interaction count : {len(filter_u_tr)}, valid interaction count : {len(filter_u_va)}, test interaction count : {len(filter_u_te)}")

u_train = set(filter_u_tr['user_id'].tolist())
u_test = set(filter_u_te['user_id'].tolist())
u_valid = set(filter_u_va['user_id'].tolist())
print(f"train user count : {len(u_train)}, valid user count : {len(u_valid)}, test user count : {len(u_test)}")

i_tr = set(filter_u_tr['recipe_id'].tolist())
i_te = set(filter_u_te['recipe_id'].tolist())
i_va = set(filter_u_va['recipe_id'].tolist())
print(f"train item count : {len(i_tr)}, valid item count : {len(i_va)}, test item count : {len(i_te)}")

i_total = i_tr|i_va|i_te
print(f"number of users to use : {len(u_total)}")
print(f"number of items to use : {len(i_total)}")

In [ ]:
import ast

train_interaction = filter_u_tr
test_interaction = filter_u_te
valid_interaction = filter_u_va
print(f"train interaction count : {len(train_interaction)}, valid interaction count : {len(valid_interaction)}, test interaction count : {len(test_interaction)}")
print(train_interaction.columns)
print(test_interaction.columns)
print(valid_interaction.columns)

recipe_processed = raw_recipe[raw_recipe['id'].isin(i_total)].reset_index(drop=True)
print(f"number of recipes to use : {len(recipe_processed)}")
print(recipe_processed.columns)

# calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV), carbohydrates (PDV)
def process_row_detailed(row):
    nutrition_types = ['calories', 'fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']
    nutrition = ast.literal_eval(row['nutrition'])
    cooking_steps = ast.literal_eval(row['steps'])

    parts = []
    for n_type, value in zip(nutrition_types, nutrition):
        if int(value) == value:
            value = int(value)
        if n_type == 'calories':
            parts.append(f"calories: {value} kcal")
        else:
            parts.append(f"{n_type}: {value}% of Daily Value")
    
    nutrition_summary = ", ".join(parts)

    parts2 = []
    for idx, step in enumerate(cooking_steps):
        parts2.append(f"{idx+1}. {step}")
    cooking_steps_summary = "\n".join(parts2)

    ingredients = ", ".join(ast.literal_eval(row['ingredients']))
    
    return pd.Series({
        'id': row['id'],
        'name': row['name'],
        'ingredients': ingredients,
        'nutrition': nutrition_summary,
        'steps' : cooking_steps_summary
    })

drop_columns_detailed = ['minutes', 'contributor_id', 'submitted', 'tags', 'n_steps',  'description', 'n_ingredients']
recipe_drop_detailed = recipe_processed.drop(columns=drop_columns_detailed)

detailed_summary_recipe = recipe_drop_detailed.apply(process_row_detailed, axis=1)
print(detailed_summary_recipe.columns)
print(detailed_summary_recipe.head(3))

# Save the preprocessed recipe textual metadata for user embedding extraction
detailed_summary_recipe.to_csv(data_path + 'recipe_textual_metadata.csv', index=False)

In [5]:
print(len(detailed_summary_recipe))
dd = pd.read_csv(data_path + 'recipe_textual_metadata.csv')
print(dd.head())

29905
      id                                               name  \
0  63986                   chicken lickin  good  pork chops   
1  43026                                     chile rellenos   
2  54100                           grilled  venison burgers   
3  25775  how i got my family to eat spinach  spinach ca...   
4  90921         i stole the idea from mirj  sesame noodles   

                                         ingredients  \
0  lean pork chops, flour, salt, dry mustard, gar...   
1  egg roll wrap, whole green chilies, cheese, co...   
2  ground venison, egg substitute, non-fat powder...   
3  frozen chopped spinach, egg, salt, black peppe...   
4  angel hair pasta, toasted sesame oil, soy sauc...   

                                           nutrition  \
0  calories: 105.7 kcal, fat: 8% of Daily Value, ...   
1  calories: 94 kcal, fat: 10% of Daily Value, su...   
2  calories: 190.9 kcal, fat: 10% of Daily Value,...   
3  calories: 166.1 kcal, fat: 16% of Daily Value,...  

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from tqdm import tqdm

recipe_ids = list(detailed_summary_recipe['id'].unique())

# =========================
# [Visual Feature] CaffeNet
# =========================
# CaffeNet ≈ AlexNet. Use torchvision's alexnet and remove the final 4096->1000 layer
try:
    from torchvision.models import AlexNet_Weights
    alexnet_weights = AlexNet_Weights.DEFAULT
except Exception:
    alexnet_weights = 'DEFAULT'

caffenet = models.alexnet(weights=alexnet_weights)
# Trim the last classification layer to get a 4096-D embedding
caffenet.classifier = torch.nn.Sequential(*list(caffenet.classifier.children())[:-1])
caffenet.eval()
caffenet.to(device)

# Use the canonical ImageNet preprocessing if available; otherwise fall back to a manual transform
try:
    transform = alexnet_weights.transforms()
except Exception:
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

batch_size = 32
image_batch = []
id_batch = []
visual_features = []
image_directory = data_path + 'images/'

for rid in tqdm(recipe_ids, desc="visual_encoding"):

    image_path1 = os.path.join(image_directory, f"{rid}.jpeg")
    image_path2 = os.path.join(image_directory, f"{rid}.png")
    image_path3 = os.path.join(image_directory, f"{rid}.jpg")
    image_path4 = os.path.join(image_directory, f"{rid}.JPG")
    image_path5 = os.path.join(image_directory, f"{rid}.PNG")

    image_path = None
    for path in [image_path1, image_path2, image_path3, image_path4, image_path5]:
        if os.path.exists(path):
            image_path = path
            break

    if not os.path.exists(image_path):
        continue
    try:
        img = Image.open(image_path).convert("RGB")
        img_tensor = transform(img)
        image_batch.append(img_tensor)
        id_batch.append(rid)
    except Exception:
        continue

    if len(image_batch) == batch_size:
        batch_tensor = torch.stack(image_batch).to(device)
        with torch.no_grad():
            batch_features = caffenet(batch_tensor).cpu().numpy()  # (N, 4096)
        for r_id, feat in zip(id_batch, batch_features):
            visual_features.append({'recipe_id': r_id, 'visual_feature': feat})
        image_batch, id_batch = [], []

# Handle leftover images
if len(image_batch) > 0:
    batch_tensor = torch.stack(image_batch).to(device)
    with torch.no_grad():
        batch_features = caffenet(batch_tensor).cpu().numpy()
    for r_id, feat in zip(id_batch, batch_features):
        visual_features.append({'recipe_id': r_id, 'visual_feature': feat})

visual_df = pd.DataFrame(visual_features)

# ===========================================
# [Textual Feature] (unchanged, per request)
# ===========================================
detailed_summary_recipe['name'] = detailed_summary_recipe['name'].fillna('')
detailed_summary_recipe['ingredients'] = detailed_summary_recipe['ingredients'].fillna('')
detailed_summary_recipe['steps'] = detailed_summary_recipe['steps'].fillna('')

detailed_summary_recipe['description'] = (
    detailed_summary_recipe['name'] + ' ' +
    detailed_summary_recipe['ingredients'] + ' ' +
    detailed_summary_recipe['steps'] + ' ' +
    detailed_summary_recipe['nutrition']
)

detailed_summary_recipe.drop(columns=['name', 'ingredients', 'nutrition', 'steps'], inplace=True)

model = SentenceTransformer('all-MiniLM-L6-v2', device = device)
descriptions = detailed_summary_recipe['description'].tolist()
recipe_ids_text = detailed_summary_recipe['id'].tolist()

# Encode with batching to avoid OOM
text_embeddings = model.encode(descriptions, batch_size=32, show_progress_bar=True)

textual_df = pd.DataFrame({
    'recipe_id': recipe_ids_text,
    'textual_feature': list(text_embeddings)
})

print(visual_df.head())
print(textual_df.head())


visual_encoding: 100%|███████████████| 29905/29905 [10:14<00:00, 48.64it/s]


Batches:   0%|          | 0/935 [00:00<?, ?it/s]

   recipe_id                                     visual_feature
0      63986  [0.0, 3.1429787, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1      43026  [0.0, 0.862355, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2      54100  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3      25775  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4      90921  [0.0, 6.0908422, 0.0, 0.0, 0.1583686, 0.0, 0.0...
   recipe_id                                    textual_feature
0      63986  [-0.10267617, -0.016740197, 0.017686494, 0.004...
1      43026  [-0.06881522, -0.017232738, -0.047181953, 0.05...
2      54100  [-0.056857362, -0.024174849, 0.0030214894, -0....
3      25775  [-0.027166935, -0.017478112, -0.07573529, -0.0...
4      90921  [-0.0876921, -0.03924904, 0.027581185, 0.03766...


In [7]:
print(len(visual_df.iloc[0]['visual_feature']))
print(len(textual_df.iloc[0]['textual_feature']))

4096
384


In [8]:
# load user ID mapping & item ID mapping
user_id_mapping = pd.read_csv(data_path + 'u_id_mapping.csv', sep='\t')
item_id_mapping = pd.read_csv(data_path + 'i_id_mapping.csv', sep='\t')

visual_dict = dict(zip(visual_df['recipe_id'], visual_df['visual_feature']))
textual_dict = dict(zip(textual_df['recipe_id'], textual_df['textual_feature']))

# === Create visual feature array ===
feat_dim_v = len(next(iter(visual_dict.values())))  # e.g., 4096
print(feat_dim_v)
visual_feat = np.zeros((len(recipe_ids), feat_dim_v))

for idx, row in item_id_mapping.iterrows():
    rid = row['recipe_id']
    visual_feat[idx] = visual_dict.get(rid, np.zeros(feat_dim_v))  # fallback if missing

np.save(data_path + 'image_feat.npy', visual_feat)
print('image_feat.npy created')

# === Create textual feature array ===
feat_dim_t = len(next(iter(textual_dict.values())))  # e.g., 384
print(feat_dim_t)
textual_feat = np.zeros((len(recipe_ids), feat_dim_t))

for idx, row in item_id_mapping.iterrows():
    rid = row['recipe_id']
    textual_feat[idx] = textual_dict.get(rid, np.zeros(feat_dim_t))  # fallback if missing

np.save(data_path + 'text_feat.npy', textual_feat)
print('text_feat.npy created')

4096
image_feat.npy created
384
text_feat.npy created


In [11]:
with open(data_path + 'foodcom.inter', 'r', encoding='utf-8') as f:
    for _ in range(10):
        print(f.readline().strip())

i_mapping = pd.read_csv(data_path + 'i_id_mapping.csv')
u_mapping = pd.read_csv(data_path + 'u_id_mapping.csv')

print("\nItem ID Mapping:")
print(i_mapping.head())

print("\nUser ID Mapping:")
print(u_mapping.head())

image_feat = np.load(data_path +'image_feat.npy')
text_feat = np.load(data_path +'text_feat.npy')

print("Image feature shape:", image_feat.shape)
print(image_feat[:3])  # show first 3 entries
print(image_feat[0][:10])
print("\nText feature shape:", text_feat.shape)
print(text_feat[:3])

userID	itemID	rating	timestamp	x_label
6	175	5	2000-10-23	0
25	445	4	2001-03-24	0
24	28	3	2001-04-02	0
21	353	4	2001-05-07	0
32	477	4	2001-05-16	0
29	438	3	2001-05-21	0
24	45	3	2001-05-25	0
25	505	4	2001-05-30	0
58	538	0	2001-06-16	0

Item ID Mapping:
  recipe_id\titemID
0             40\t0
1             49\t1
2             58\t2
3             66\t3
4            142\t4

User ID Mapping:
  user_id\tuserID
0         1535\t0
1         1634\t1
2         1676\t2
3         1891\t3
4         2586\t4
Image feature shape: (29905, 4096)
[[0.         8.43509769 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[0.         8.43509769 0.         0.         0.0208737  0.
 0.         0.         0.         1.61045408]

Text feature shape: (29905, 384)
[[-0.08076137 -0.02005294 -0.0355938  ... -0.01922152 -0.04684478
  -0.07948707]
 [-0.0339156  -0.02230442  